
<div align="center">
<h2 style="text-align: center;" markdown="1">Journées Mathrice</h2>
<img alt="Mathrice" style="border-width:0" width="15%" src="images/logo_mathrice.png">
<h3 style="text-align: center;" markdown="1">Montpellier, 27-29 mars 2018</h3>
</div>                                       
<hr>
<h1 style="text-align: center;" markdown="0">Publier sur math.cnrs.fr</h1>
<h1 style="text-align: center;" markdown="0">avec GitLab Pages</h1>
<br>
<div align="center">
<a rel="diaporama" href="http://mboileau.pages.math.cnrs.fr/gitlab-pages/">Diaporama reveal.js</a> -
<a rel="html" href="http://mboileau.pages.math.cnrs.fr/gitlab-pages/gitlab-pages.html">page html</a>
</div>                                       
<hr>
<h4 style="text-align: center;" markdown="0"><em>Matthieu Boileau</em></h4>
<h4 style="text-align: center;" markdown="0"><em>IRMA, CNRS, Université de Strasbourg</em></h4>
<hr>


<div align="center">
<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Licence Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png"/></a>
</div>

![](images/stacked_wm_no_bg_small.png)

# GitLab Pages

À partir de projets hébergés sur <https://plmlab.math.cnrs.fr>, GitLab Pages permet de :

- publier un **site web statique**, construit par exemple avec un générateur de site statique,
- mettre en ligne des **pdf** d'articles de livres, de cours, d'énoncé d'exercices, etc., construits à partir de sources Tex, par exemple.
- mettre en ligne des **diaporamas**, comme celui-ci...



<img alt="Settings" style="border-width:0" width="10%" src="images/pages_group_avatar.png">

# Du point de vue l'utilisateur

## Permet de publier sur une URL en `math.cnrs.fr`

Pages permet de publier sur une URL qui contient un nom de projet ou d'équipe.

Pour le projet GitLab `projectname` qui appartient à
`username` ou `groupname`, l'adresse de publication sera :

```
https://[username|groupname].pages.math.cnrs.fr/[projectname]
```

> `projectname` est absent de l'URL s'il vaut `[username|groupname].pages.math.cnrs.fr`

## Permet de faire de l'édition collaborative

- Utiliser Git et GitLab pour **enregistrer et fusionner** les différentes contributions
- Utiliser GitLab Pages pour **reconstruire** le site ou le document en ligne après chaque modification sur le dépôt GitLab

## Très facile à mettre en oeuvre

En deux temps, trois mouvements :

1. On enregistre un `gitlab runner` ou on utilise un *runner* partagé
2. On ajoute un fichier `.gitlab-ci.yml` décrivant le job d'intégration continue

# Limitations/parades

- on ne peut pas héberger de contenu web dynamique **mais** les générateurs de sites statiques font déjà beaucoup !
- une taille maximale de contenu publiable (100Mo par défaut) **mais** peut être étendu par l'admin
- l'URL de publication contient forcément `.pages.math.cnrs.fr` **mais** si l'admin l'active, on peut [ajouter un nom de domaine personnalisé](https://docs.gitlab.com/ee/user/project/pages/introduction.html#add-a-custom-domain-to-your-pages-website).

<img alt="Settings" style="border-width:0" width="10%" src="images/pages_group_avatar.png">

# Du point de vue de l'administrateur

## Avantages

- aucune intervention nécessaire de la part de l'admin : la publication est automatiquement déclenchée par le job d'intégration continue
- on évite de gérer à la main "un site web => un serveur web (ou un virtualhost)" : Pages le fait automatiquement !
- pas de problème de sécurité lié à du contenu dynamique : l'exécution de code (php ou autre) n'est pas autorisée.

<img alt="Settings" style="border-width:0" width="10%" src="images/gear.png">

## Configuration

Quelques lignes dans le fichier `/etc/gitlab/gitlab.rb` :

```ruby
################
# gitlab pages #
################
pages_external_url 'https://pages.math.cnrs.fr'

pages_nginx['redirect_http_to_https'] = true
pages_nginx['ssl_certificate'] = "/etc/gitlab/ssl/star_pages_math_cnrs_fr.pem"
pages_nginx['ssl_certificate_key'] = "/etc/gitlab/ssl/star_pages_math_cnrs_fr.key"
```

où `star_pages_math_cnrs_fr.[pem|key]` sont les fichiers du certificat wildcard qui est nécessaire pour publier sur `*.pages.math.cnrs.fr`.

## Précautions

- il est très facile passer du contenu privé (projet gitlab) à du contenu public (Pages)
- ne publier que la branche master
- s'assurer que les utilisateurs ont adhéré à une charte

# Démonstration

## Etape 1 : enregistrement d'un *gitlab runner*

GitLab Pages repose sur le workflow d'intégration continue de GitLab : gitlab-ci.
Avec gitlab-ci, les jobs d'intégration continue sont assurés par les *gitlab runners*.

Pour plus de souplesse, on enregistre un *gitlab runner* de type `Docker` en suivant [la doc officielle](https://docs.gitlab.com/runner/commands/README.html#gitlab-runner-register).

- On récupère le token d'intégration continue du projet sur la page du projet dans 

```
Settings > CI / CD > Runners settings
```

- On positionne la variable d'environnement correspondante (déjà fait pour les besoins de cette présentation)

In [ ]:
#export REGISTRATION_TOKEN=my-registration-token

- On enregistre le runner

In [ ]:
gitlab-runner register --non-interactive \
                       --name gl_runnner_for_$USER \
                       --url "https://plmlab.math.cnrs.fr" \
                       --executor docker \
                       --docker-image boileaum/jupyter

`--docker-image boileaum/jupyter` indique que le conteneur Docker qui sera chargé d'exécuter le script d'intégration continue sera instancié à partir de l'image [`boileaum/jupyter`](https://hub.docker.com/r/boileaum/jupyter/)

- On vérifie que le *runner* est actif

In [ ]:
gitlab-runner list

## Etape 2 : configuration du fichier `gitlab-ci.yml`



In [ ]:
cat > .gitlab-ci.yml <<- EOM
pages:
  script:
    - pip install -r requirements.txt
    - ./convert.sh
    - mv gitlab-pages.slides.html index.html
    - mkdir public
    - cp index.html gitlab-pages.html gitlab-pages.ipynb public/
    - cp -r images public/
  only:
    - pages
  artifacts:
    paths:
      - public
EOM

avec :

- `pages:` le job gitlab-ci doit obligatoirement porter ce nom
- `convert.sh` un script qui convertit le diaporama Reveal.js 
- `public/` le répertoire dont le contenu sera publié par Pages

```yaml
  only:
    - pages
```

Ce job ne sera exécuté que pour la branche `pages`.

```yaml
  artifacts:
    paths:
      - public
```

Nécessaire pour que le répertoire soit effectivement publié.

# Conclusion

- GitLab Pages facilite la publication de contenu : sites web, documentation de code, articles, support de cours, d'exposés, etc.
- Pages fait partie de la chaîne d'intégration continue de GitLab
- gitlab-ci permet de gérer la construction du contenu html à partir de sources
- GitLab Pages peut être une façon de collaborer sur un site sans besoin de CMS

# Ressources

- Documentation officielle très complète
- exemples
- tutoriels